In [1]:
import sys
import cv2 as cv
import numpy as np
#  Global Variables
DELAY_CAPTION = 1500
DELAY_BLUR = 100
MAX_KERNEL_LENGTH = 31
src = None
dst = None
window_name = 'Smoothing Demo'
def main(argv):
    cv.namedWindow(window_name, cv.WINDOW_AUTOSIZE)
    #Load the source image
    global src
    src = cv.imread("lena.jpg")
    if src is None:
        print ('Error opening image')
        print ('Usage: smoothing.py [image_name -- default ../data/lena.jpg] \n')
        return -1
    if display_caption('Original Image') != 0:
        return 0
    global dst
    dst = np.copy(src)
    if display_dst(DELAY_CAPTION) != 0:
        return 0
    # Applying Homogeneous blur
    print("Homogeneous Blur Values:")
    if display_caption('Homogeneous Blur') != 0:
        return 0
    
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.blur(src, (i, i))
        if display_dst_psnr(dst, DELAY_BLUR) != 0:
            return 0 
    
    # Applying Gaussian blur
    print("Gaussian Blur Values:")
    if display_caption('Gaussian Blur') != 0:
        return 0
    
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.GaussianBlur(src, (i, i), 0)
        if display_dst_psnr(dst, DELAY_BLUR) != 0:
            return 0
    
    # Applying Median blur
    print("Media Blur Values:")
    if display_caption('Median Blur') != 0:
        return 0
    
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.medianBlur(src, i)
        if display_dst_psnr(dst, DELAY_BLUR) != 0:
            return 0
    
    # Applying Bilateral Filter
    print("Bilateral Filter Values:")
    if display_caption('Bilateral Blur') != 0:
        return 0
    
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.bilateralFilter(src, i, i * 2, i / 2)
        if display_dst_psnr(dst, DELAY_BLUR) != 0:
            return 0
    #  Done
    display_caption('Done!')
    return 0

# Peak Signal-to-Noise Ratio (PSNR)
# Link: https://www.ni.com/en/shop/data-acquisition-and-control/add-ons-for-data-acquisition-and-control/what-is-vision-development-module/peak-signal-to-noise-ratio-as-an-image-quality-metric.html#:~:text=The%20term%20peak%20signal-to,the%20quality%20of%20its%20representation.
def get_psnr(original, processed):
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        psnr = float('inf')
    else:
        max_pixel = 255.0
        psnr = 10 * np.log10((max_pixel ** 2) / mse)
    print("PSNR:", psnr) 

def display_caption(caption):
    global dst
    dst = np.zeros(src.shape, src.dtype)
    rows, cols, _ch = src.shape
    cv.putText(dst, caption,
                (int(cols / 4), int(rows / 2)),
                cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255))
    return display_dst(DELAY_CAPTION)
def display_dst(delay):
    cv.imshow(window_name, dst)
    c = cv.waitKey(delay)
    if c >= 0 : return -1
    return 0
# display_dst but with the added get_psnr
def display_dst_psnr(image, delay):
    get_psnr(src, image)
    return display_dst(delay)

if __name__ == "__main__":
    main(sys.argv)

Homogeneous Blur Values:
PSNR: inf
PSNR: 35.45162623354163
PSNR: 33.669873948913754
PSNR: 32.81556016834722
PSNR: 32.22935304180885
PSNR: 31.78900696863898
PSNR: 31.438232888777897
PSNR: 31.154005446033995
PSNR: 30.911495534186592
PSNR: 30.685074306326065
PSNR: 30.500217880148885
PSNR: 30.317076201210888
PSNR: 30.15487268475438
PSNR: 30.00955000443412
PSNR: 29.873956847181216
Gaussian Blur Values:
PSNR: inf
PSNR: 36.77200663957023
PSNR: 35.16650061859132
PSNR: 34.0780121122748
PSNR: 33.53641177332305
PSNR: 33.11349015372436
PSNR: 32.73363938629351
PSNR: 32.45340953384927
PSNR: 32.1941674685762
PSNR: 31.963092546736508
PSNR: 31.781760970810353
PSNR: 31.587659807318282
PSNR: 31.430833274313365
PSNR: 31.279192886465598
PSNR: 31.147064160112127
Media Blur Values:
PSNR: inf
PSNR: 36.87849397534908
PSNR: 34.790717235794354
PSNR: 33.92401171406962
PSNR: 33.371664384338914
PSNR: 32.953226690193794
PSNR: 32.62740393111075
PSNR: 32.35965786673968
PSNR: 32.12168793781039
PSNR: 31.91457535588362
P

Which method had the best score? Why do you think this denoising method performed the best (explain in terms of its inherent steps / properties)? 

Based on the returned PSNR scores, the Bilateral Filter method performed the best when it came to preserving image fidelity and having reduced distortions. This may be due to the Bilateral Filter's approach of assigning weights to neighboring pixels. Unlike the Gaussian filter, which just assigns a weight to neighboring pixels, the Bilateral Filter also considers spatial and intensity differences when assigning weights. The weight having two components allows it to preserve edges effectively by reducing the tendency of edges to be smoothed away during the denoising process. The filter substitutes the bright pixel at its center with the average of nearby bright pixels, while ignoring the dark pixels. On the contrary, if the filter is centered over a dark pixel, it ignores the bright pixels and focuses on the dark ones.

Bilateral Filtering for Gray and Color Images: https://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MANDUCHI1/Bilateral_Filtering.html